<a href="https://colab.research.google.com/github/cufuentes/CodingDojo/blob/Week-3/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [2]:
filename = "/content/insurance.csv"
df = pd.read_csv(filename)
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
y = df["charges"]
X = df.drop(columns = "charges")

In [6]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state =42)
X_train

,age,sex,bmi,children,smoker,region
560,46,female,19.950,2,no,northwest
1285,47,female,24.320,0,no,northeast
1142,52,female,24.860,0,no,southeast
969,39,female,34.320,5,no,southeast
486,54,female,21.470,3,no,northwest
...,...,...,...,...,...,...
1095,18,female,31.350,4,no,northeast
1130,39,female,23.870,5,no,southeast
1294,58,male,25.175,0,no,northeast
860,37,female,47.600,2,yes,southwest


Age: Ordinal
sex: nominal
bmi: Ordinal
children: Ordinal
Smoker: nominal
region: nominal

Selecccionar categorias y One Hot Encoding

In [9]:
cat_selector = make_column_selector(dtype_include ="object")
cat_selector(X_train)

['sex', 'smoker', 'region']

In [10]:
train_cat_data = X_train[cat_selector(X_train)]
test_cat_data = X_test[cat_selector(X_test)]

,sex,smoker,region
560,female,no,northwest
1285,female,no,northeast
1142,female,no,southeast
969,female,no,southeast
486,female,no,northwest


In [11]:
ohe_encoder = OneHotEncoder (sparse_output = False, handle_unknown ="ignore")
ohe_encoder.fit(train_cat_data)
ohe_columns_names = ohe_encoder.get_feature_names_out(train_cat_data.columns)

train_ohe = ohe_encoder.transform (train_cat_data)
test_ohe = ohe_encoder.transform(test_cat_data)

train_ohe = pd.DataFrame(train_ohe, columns = ohe_columns_names)
test_ohe = pd.DataFrame(test_ohe, columns = ohe_columns_names)
train_ohe.head()

,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


Escalar caracteristicas numericas

In [14]:
num_selector = make_column_selector(dtype_include ="number")
train_num_data = X_train[num_selector(X_train)]
test_num_data = X_test[num_selector(X_test)]
train_num_data.head()

,age,bmi,children
560,46,19.95,2
1285,47,24.32,0
1142,52,24.86,0
969,39,34.32,5
486,54,21.47,3


In [15]:
scaler = StandardScaler()
scaler.fit(train_num_data)

StandardScaler()

In [16]:
train_scaled = scaler.transform(train_num_data)
test_scaled = scaler.transform (test_num_data)
train_scaled[:5]

array([[ 0.47222651, -1.75652513,  0.73433626],
       [ 0.54331294, -1.03308239, -0.91119211],
       [ 0.8987451 , -0.94368672, -0.91119211],
       [-0.02537852,  0.62239299,  3.20262882],
       [ 1.04091797, -1.50489287,  1.55710045]])

In [17]:
train_scaled = pd.DataFrame (train_scaled , columns = train_num_data.columns)
test_scaled = pd.DataFrame (test_scaled, columns = test_num_data.columns)
train_scaled.head()

,age,bmi,children
0,0.472227,-1.756525,0.734336
1,0.543313,-1.033082,-0.911192
2,0.898745,-0.943687,-0.911192
3,-0.025379,0.622393,3.202629
4,1.040918,-1.504893,1.557100


Reseteamos Index y concatenamos los df

In [18]:
train_scaled = train_scaled.reset_index(drop=True)
test_scaled = test_scaled.reset_index(drop=True)
X_train_processed = pd.concat([train_ohe, train_scaled],axis=1)
X_test_processed = pd.concat([test_ohe,test_scaled],axis=1)
X_train_processed.head()

,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,age,bmi,children
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.472227,-1.756525,0.734336
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.543313,-1.033082,-0.911192
2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.898745,-0.943687,-0.911192
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.025379,0.622393,3.202629
4,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.040918,-1.504893,1.557100
